In [1]:
import glob
import os
import sys
import platform
import requests
import batanalysis as ba
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time, TimeDelta
from astropy.io import fits
from pathlib import Path
import swiftbat.swutil as sbu
import swiftbat
plt.ion()

ba.datadir("/home/idies/workspace/Temporary/ashleyh12/scratch/grb240529A")


PosixPath('/home/idies/workspace/Temporary/ashleyh12/scratch/grb240529A')

In [2]:
# handling query database issue
from astroquery.simbad import Simbad

try: 
    x = Simbad.query_object("GRB240529A")
    if x is not None: # if x != None:
        print(f'{x}')
    else:
        print("NOT FOUND")
        
except Exception as y:   
    print("")

NOT FOUND


In [3]:
grb_name = "GRB240529A"


In [4]:
#function to retrieve XRT RA and DEC

def XRTcoord(grb_identifier):
    
    # url of the file
    url = "https://swift.gsfc.nasa.gov/archive/grb_table/tmp/grb_table_1720641542.txt"
    
    #sending a https get request to the url above
    response = requests.get(url)
    
    # extract the data from the url (.text to handle text-based responses)
    table_data = response.text.splitlines()
    
    #initializing variables
    XRTra = None
    XRTdec = None
    
    # iterating over the data
    for line in table_data:
        columns = line.split() # columns
        if columns[0] == grb_identifier and len(columns) > 10:
                XRTra = columns[14]
                XRTdec = columns[15]
                return XRTra, XRTdec




grb_identifier = "240529A"
print(XRTcoord(grb_identifier))
XRTra, XRTdec = XRTcoord(grb_identifier)


('22:21:25.86', '51:33:42.3')


In [5]:
# conversion: Degrees(h,m,s) = 360/24 × (h + m/60 + s/3600) 
# Link: (https://www.reddit.com/r/astrophysics/comments/wsznci/comment/il23y2i/?utm_medium=android_app&utm_source=share&context=3)

# XRTra_degrees = 360/24 * (int(XRTra[0:2]))
# print(XRTra_degrees)

from astropy.coordinates import SkyCoord
from astropy import units as u
c = SkyCoord(ra=XRTra, dec=XRTdec, unit=(u.hourangle, u.deg))
XRTra_deg = c.ra.degree
XRTdec_deg = c.dec.degree
print(XRTra_deg, XRTdec_deg)

335.35775 51.561749999999996


In [6]:
object_name = "GRB240529A"
object_batsource = swiftbat.source(
    ra=XRTra_deg, dec=XRTdec_deg, name=grb_identifier
)
print(object_batsource)

|||240529A||0||||335.357750|51.561750|


In [7]:
# automating the grb name format using astropy
# CONVERTING THE GRB NAME TO A MONTH-YEAR-DAY FORMAT

from astropy.time import Time

def grb_date(grb_name):
    year = "20" + grb_name[3:5]  # converting the grb name to years
    month = grb_name[5:7] # converting the name to months via indexing
    day = grb_name[7:9]  # converting it to days 

    date_format = f"{year}-{month}-{day}" # setting up the string format
    time = Time(date_format, format='iso') # using the Astropy Time object and passing the date string into it
    return time
    
grb_name = "GRB240529A"
grb_time = grb_date(grb_name) # storing the result of time format from above function
last_day = grb_time + 7 # adding a week to the above time 
print(f"{str(grb_time)} .. {str(last_day)}")

2024-05-29 00:00:00.000 .. 2024-06-05 00:00:00.000


In [8]:
query_args = dict(Start_Time=str(grb_date(grb_name)), fields='All', resultmax=0)
table_everything = ba.from_heasarc(**query_args)
minexposure = 1000     # cm^2 after cos adjust

# object_batsource = swiftbat.source(ra=XRTra_deg, dec=36.300, name=grb_name)

#calculate the exposure with partial coding
exposures = np.array([object_batsource.exposure(ra=row['RA'], dec=row['DEC'], roll=row['ROLL_ANGLE'])[0] for row in table_everything])

#select the observations that have greater than the minimum desired exposure
table_exposed = table_everything[exposures > minexposure]
print(f"Finding everything finds {len(table_everything)} observations, of which {len(table_exposed)} have more than {minexposure:0} cm^2 coded")

print(np.sort(table_exposed["START_TIME"]))

Finding everything finds 126 observations, of which 23 have more than 1000 cm^2 coded
   START_TIME   
      mjd       
----------------
 60458.339537037
 60459.377037037
 60459.652037037
60458.0853819444
60458.0888425926
60458.1360648148
60458.1888541667
60458.2291319444
60458.2728703704
60458.5839930556
             ...
60459.1214814815
60459.1902314815
60459.3152314815
60459.4028819444
60459.4638425926
60459.4646643518
60459.4735763889
60459.5368171296
60459.7173148148
60459.9168055556
Length = 23 rows


In [9]:
# get a list of the fully downloaded observation IDs
result = ba.download_swiftdata(table_exposed)

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: SWIFT_TLE_ARCHIVE.txt.24158.60038926.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488001pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488001pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488001pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488001ppr.par.gz exists and not ov

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488002pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488002pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488002pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488002ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw01231488002s.mkf.gz exists and not overwritten (set clo

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00090372003pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00090372003pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00090372003pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00090372003ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00090372003s.mkf.gz exists and not overwritten (set clo

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097669003pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097669003pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097669003pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097669003ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097669003s.mkf.gz exists and not overwritten (set clo

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097641001pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097641001pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097641001pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097641001ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097641001s.mkf.gz exists and not overwritten (set clo

...

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00032035190pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00032035190pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00032035190pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00032035190ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00032035190s.mkf.gz exists and not overwritten (set clo

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074003057pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074003057pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074003057pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074003057ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074003057s.mkf.gz exists and not overwritten (set clo

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097612001pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097612001pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097612001pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097612001ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00097612001s.mkf.gz exists and not overwritten (set clo

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03112289002pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03112289002pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03112289002pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03112289002ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03112289002s.mkf.gz exists and not overwritten (set clo

/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074010093pat.fits.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074010093pjb.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074010093pob.cat.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074010093ppr.par.gz exists and not overwritten (set clobber=True to override this).
/home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00074010093s.mkf.gz exists and not overwritten (set clo

In [10]:
final_obs_ids = [i for i in table_exposed['OBSID'] if result[i]['success']]
print(len(final_obs_ids))

23


In [11]:
from astropy.coordinates import SkyCoord

# object_name="GRB240529A"
# object_location = swiftbat.simbadlocation(object_name)
# c=SkyCoord(ra=object_location[0], dec=object_location[1], unit="deg", frame="icrs")

incat=ba.create_custom_catalog(object_name, c.ra.value, c.dec.value, c.galactic.l.value, c.galactic.b.value, catalog_dir=ba.datadir())
dir_path = "/home/idies/workspace/Temporary/ashleyh12/scratch/grb240529A/"
incat = Path(f"{ba.datadir(dir_path)}/custom_catalog.cat")
print(incat)

/home/idies/workspace/Temporary/ashleyh12/scratch/grb240529A/custom_catalog.cat


In [ ]:
#run batsurvey in parallel
input_dict=dict(cleansnr=6,cleanexpr='ALWAYS_CLEAN==T', incatalog=f"{incat}", detthresh=9000, detthresh2=9000)
noise_map_dir = Path("/home/idies/workspace/Temporary/tmpataki/scratch/PATTERN_MAPS/")

batsurvey_obs=ba.parallel.batsurvey_analysis(final_obs_ids,  input_dict=input_dict, patt_noise_dir=noise_map_dir,  nprocs=10, recalc=True)
# batsurvey_obs=ba.parallel.batsurvey_analysis(final_obs_ids,  input_dict=input_dict)


In [13]:
# %debug

> /home/idies/miniconda3/envs/heasoft/lib/python3.10/site-packages/batanalysis/bat_survey.py(259)__init__()
    257 
    258                     # find the date closest to the time of the observation start time
--> 259                     idx = np.abs(time - patt_dates).argmin()
    260 
    261                     # save the name



ipdb>  time


numpy.datetime64('2024-05-29T04:36:15.467955018')


ipdb>  patt_dates


array([], dtype=float64)


ipdb>  print(all_patt_map)


[]


ipdb>  print(patt_noise_dir)


/home/idies/workspace/Temporary/ashleyh12/scratch/PATTERN_MAPS


ipdb>  quit


In [ ]:
# batsurvey_obs=ba.parallel.batsurvey_analysis(final_obs_ids,  input_dict=input_dict, patt_noise_dir=map_dir)

In [ ]:
# batsurvey_obs=ba.parallel.batspectrum_analysis(batsurvey_obs, object_name, fit_iterations=1000)

batsurvey_obs = ba.parallel.batspectrum_analysis(
    batsurvey_obs, object_name, use_cstat=False, ul_pl_index=2, nprocs=6
)

In [ ]:
ba.print_parameters(batsurvey_obs, source_name, values=['met_time', 'utc_time', 'exposure', 'flux', 'index'], latex_table=False, savetable=True, save_file="output.txt")

In [ ]:
# fig, axes=ba.plot_survey_lc(batsurvey_obs, id_list=source_name, time_unit="UTC", values=["rate","snr", "flux", "exposure"], T0=306757, calc_lc=True) #T0 is GBM trigger time

fig, axes=ba.plot_survey_lc(batsurvey_obs, id_list=source_name, time_unit="UTC", values=["rate","snr", "flux", "exposure"]) #T0 is GBM trigger time

axes[1].set_yscale('log')
axes[0].set_yscale('log')


axes[1].axhline(5,0,1)

fig.savefig('grb240529A_lightcurve.pdf')

In [ ]:
#combine all the pointings into a single file to sort into binned fits files
outventory_file=ba.merge_outventory(batsurvey_obs)

In [ ]:
#bin into 1 day cadence
time_bins=ba.group_outventory(outventory_file, np.timedelta64(1, "D"), end_datetime=Time("2008-03-21"))

In [ ]:
#do the parallel construction of each mosaic for each time bin
time_bins=ba.group_outventory(outventory_file, np.timedelta64(1, "D"), end_datetime=Time("2008-03-21"))
mosaic_list, total_mosaic=ba.parallel.batmosaic_analysis(batsurvey_obs, outventory_file, time_bins, catalog_file=incat, nprocs=1)

In [ ]:
mosaic_list=ba.parallel.batspectrum_analysis(mosaic_list, source_name, fit_iterations=1000, nprocs=1)
total_mosaic =ba.parallel.batspectrum_analysis(total_mosaic, source_name, fit_iterations=1000, nprocs=1)

In [ ]:
fig, axes=ba.plot_survey_lc(mosaic_list, id_list=source_name, time_unit="UTC", values=["rate","snr", "flux", "exposure"], T0=687014249.000, calc_lc=True) #T0 is GBM trigger time

axes[1].set_yscale('log')
axes[0].set_yscale('log')


axes[1].axhline(5,0,1)

ba.print_parameters(mosaic_list, source_name, values=['met_time', 'utc_time', 'exposure', 'flux', 'PhoIndex'], latex_table=False, savetable=True, save_file="output_mosaic.txt")

ba.print_parameters([total_mosaic], source_name, values=['met_time', 'utc_time', 'exposure', 'flux', 'PhoIndex'], latex_table=False, savetable=True, save_file="output_mosaic_total.txt")

energy_range=None
time_unit="MET"
values=["rate","snr", "flux", "PhoIndex"]

survey_obsid_list=["all_data_dictionary","monthly_mosaic_dictionary"]

# obs_list_count=0
for observation_list in survey_obsid_list:

    with open(observation_list+".pkl", 'rb') as f:
        all_data=pickle.load(f)
        data=all_data[object_name]

    # get the time centers and errors
    if "mosaic" in observation_list:

        if "MET" in time_unit:
            t0 = TimeDelta(data["user_timebin/met_time"], format='sec')
            tf = TimeDelta(data["user_timebin/met_stop_time"], format='sec')
        elif "MJD" in time_unit:
            t0 = Time(data[time_str_start], format='mjd')
            tf = Time(data[time_str_end], format='mjd')
        else:
            t0 = Time(data["user_timebin/utc_time"])
            tf = Time(data["user_timebin/utc_stop_time"])
    else:
        if "MET" in time_unit:
            t0 = TimeDelta(data["met_time"], format='sec')
        elif "MJD" in time_unit:
            t0 = Time(data[time_str_start], format='mjd')
        else:
            t0 = Time(data["utc_time"])
        tf = t0 + TimeDelta(data["exposure"], format='sec')

    dt = tf - t0

    if "MET" in time_unit:
        time_center = 0.5 * (tf + t0).value
        time_diff = 0.5 * (tf - t0).value
    elif "MJD" in time_unit:
        time_diff = 0.5 * (tf - t0)
        time_center = t0 + time_diff
        time_center = time_center.value
        time_diff = time_diff.value

    else:
        time_diff = TimeDelta(0.5 * dt)  # dt.to_value('datetime')
        time_center = t0 + time_diff

        time_center = np.array([i.to_value('datetime64') for i in time_center])
        time_diff = np.array([np.timedelta64(0.5 * i.to_datetime()) for i in dt])

    x = time_center
    xerr = time_diff

    if obs_list_count == 0:
        fig, axes = plt.subplots(len(values), sharex=True, figsize=(10,12))

    axes_queue = [i for i in range(len(values))]
    # plot_value=[i for i in values]

    e_range_str = f"{14}-{195} keV"
    #axes[0].set_title(object_name + '; survey data from ' + e_range_str)

    for i in values:
        ax = axes[axes_queue[0]]
        axes_queue.pop(0)

        y = data[i]
        yerr = np.zeros(x.size)
        y_upperlim = np.zeros(x.size)

        label = i

        if "rate" in i:
            yerr = data[i + "_err"]
            label = "Count rate (cts/s)"
        elif i + "_lolim" in data.keys():
            # get the errors
            lolim = data[i + "_lolim"]
            hilim = data[i + "_hilim"]

            yerr = np.array([lolim, hilim])
            y_upperlim = data[i + "_upperlim"]

            # find where we have upper limits and set the error to 1 since the nan error value isnt
            # compatible with upperlimits
            yerr[:, y_upperlim] = 0.1 * y[y_upperlim]

        if "mosaic" in observation_list:
            if "weekly" in observation_list:
                zorder = 9
                c = "blue"
                m = "o"
                l="Weekly Mosaic"
                ms=5
                a=0.8
            if "daily" in observation_list:
                zorder = 9
                c = "gray"
                m = "o"
                l="Daily Mosaic"
                ms=5
                a=0.8
            else:
                zorder = 9
                c='green'
                m = "s"
                l = "Monthly Mosaic"
                ms=7
                a = 1
        else:
            zorder = 4
            c = "r"
            m = "o"
            l = "Survey Snapshot"
            ms=3
            a = 1

        ax.errorbar(x, y, xerr=xerr, yerr=yerr, uplims=y_upperlim, linestyle="None", marker=m, markersize=ms,
                    zorder=zorder, color=c, label=l, alpha=a)

        if ("flux" in i.lower()):
            ax.set_yscale('log')

        if ("snr" in i.lower()):
            ax.set_yscale('log')

        ax.set_ylabel(label)

    # if T0==0:
    if "MET" in time_unit:
        label_string = 'MET Time (s)'
        plt.gca().ticklabel_format(useMathText=True)
    elif "MJD" in time_unit:
        label_string = 'MJD Time (s)'
    else:
        label_string = 'UTC Time (s)'

    axes[-1].set_xlabel(label_string)

    obs_list_count += 1

axes[0].set_yscale('log')
axes[1].set_yscale('log')
axes[1].axhline(7,0,1)


#add the UTC times as well
met_values=[687014249.000]#[i.get_position()[0] for i in axes[-1].get_xticklabels()]
utc_values=[np.datetime64(sbu.met2datetime(i)) for i in met_values]

if "MET" in time_unit:
    plot_val=met_values
elif "UTC" in time_unit:
    plot_val=utc_values

for i,j in zip(plot_val, ["19-03-08 06:12:49 UTC"]):
    for ax in axes:
        ax.axvline(i, 0, 1, ls='--', color='k')
        if ax==axes[0]:
            ax.text(i, ax.get_ylim()[1]*1.03, str(j), fontsize=12, ha='center')

axes[0].legend(loc="upper left")

axes[1].set_ylabel("SNR")
axes[2].set_ylabel(r"Flux (erg/s/cm$^2$)")
axes[3].set_ylabel(r"$\Gamma$")

for ax, l in zip(axes, ["a","b","c","d"]):
    ax.text(.99, .95, f"({l})", ha='right', va='top', transform=ax.transAxes,  fontsize=12)

#crab values for reference
axes[-1].axhline(2.15, 0, 1)
axes[-2].axhline(23342.70e-12, 0, 1)

fig.tight_layout()
plot_filename = object_name + '_survey_lc.pdf'
fig.savefig(plot_filename, bbox_inches="tight")

In [ ]:
# import sys
# import platform

In [ ]:
# #run batsurvey in parallel
# input_dict=dict(cleansnr=6,cleanexpr='ALWAYS_CLEAN==T', incatalog=f"{incat}", detthresh=9000, detthresh2=9000)
# map_dir = Path("/home/idies/workspace/Temporary/tmpataki/scratch/PATTERN_MAPS/")
# obs=ba.BatSurvey(final_obs_ids[0],recalc=True,  input_dict=input_dict, patt_noise_dir=map_dir)

In [ ]:
# %debug